# WIP
## IES - Python - Project
### Marathon Results Analysis
#### David Koubek, Jiri Zelenka

#### Import required packages.

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from IPython.core.debugger import Tracer

### Robots.txt

Are we allowed to scrape?

In [2]:
requests.get('https://www.runczech.com/robots.txt')

<Response [200]>

The response 200 means the request was fulfilled. Let's look visually at the actual robots.txt file what is allowed and what's not.

In [3]:
print(requests.get('https://www.runczech.com/robots.txt').text)

#
# robots.txt
#

# exclude these directories
User-agent: *
Disallow: /srv/
Disallow: /cgi/
Allow: /srv/www/qf/*/ramjet/eventList
Allow: /srv/www/qf/*/ramjet/eventVoucherList
Allow: /srv/www/qf/*/ramjet/contactPage
Allow: /srv/www/qf/*/ramjet/raceDetail
Allow: /srv/www/qf/*/ramjet/leagueDetail
Allow: /srv/www/qf/*/ramjet/results/list
Allow: /srv/www/qf/*/ramjet/results/league
Allow: /srv/www/qf/*/ramjet/results/league/detail
Allow: /srv/www/qf/*/ramjet/resultsEventDetail
Allow: /srv/www/qf/*/ramjet/resultsSubEventUserDetail
Allow: /srv/www/qf/*/ramjet/resultsSubEventGroupDetail
Allow: /srv/www/qf/*/ramjet/event/runnerList

Sitemap: https://www.runczech.com/sitemap-cs.xml
Sitemap: https://www.runczech.com/sitemap-en.xml
Sitemap: https://www.runczech.com/sitemap-de.xml
Sitemap: https://www.runczech.com/sitemap-it.xml
Sitemap: https://www.runczech.com/sitemap-fr.xml
Sitemap: https://www.runczech.com/sitemap-es.xml
Sitemap: https://www.runczech.com/sitemap-pl.xml
Sitemap: https://www.runcz

The "resultsEventDetail" which we desire to scrape is allowed which is good, we can proceed.

## Pages

In [4]:
def getSoup(link):
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml')

### Scrape the whole HTML of a single link

In [5]:
def getAllLinks(link):
    soup = getSoup(link)

    return soup # temporarily return just the whole HTML to make sure at least this works
    
marathons_links = getAllLinks('https://www.runczech.com/srv/www/qf/en/ramjet/results/list?&page=1&per_page=15')

In [6]:
marathons_links

<!DOCTYPE html>
<!--[if lte IE 7 ]><html class="nojs ie ie7" lang="en"><![endif]--><!--[if IE 8 ]><html class="nojs ie ie8" lang="en"><![endif]--><!--[if IE 9 ]><html class="nojs ie ie9" lang="en"><![endif]--><!--[if !IE]><!--><html class="nojs" lang="en"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>Results | RunCzech</title>
<meta content="index, follow" name="robots"/>
<meta content="index, follow, snippet, archive" name="googlebot"/>
<meta content="FG Forrest, a.s., www.fg.cz, 2013" name="author"/>
<meta content="" name="keywords"/>
<meta content="" name="description"/>
<link href="https://cdn-vsh.runczech.com/favicon.ico#v=2018" rel="shortcut icon" type="image/x-icon"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/en/rss/news.xml" rel="alternate" title="RunCzech - News" type="application/rss+xml"/>
<link href="https://cdn-vsh.runczech.com/ver/20180712092409/css/u/fg-skeleton.css" media="all" rel="stylesheet" type="text/css"/>
<link 

No middlepage table can be found in output as it's loaded via javascript. The eventID is pulled from somewhere (their servers, JSON) only upon loading the page using their script.

This is inside of the script that has the crucial eventId='+item.id+' that is not statically in the HTML:

```<a class="indexList_link name" href="/srv/www/qf/en/ramjet/resultsEventDetail?eventId='+item.id+'">```
    
Once the webpage is loaded in the browser, this is how the link we seek looks like, already the eventID is rendered to the code:

```<a class="indexList_link name" href="/srv/www/qf/en/ramjet/resultsEventDetail?eventId=22175">Mattoni Karlovy Vary Half Marathon - 21.0975 km 2019</a>```

See how the browser renders the eventID into the HTML using the script (links_tag_js_issue.png):

<img src="v0-links_tag_js_issue.png">

### Find all marathon links

Proceed to check programmatically that we cannot pull any links from the middlepage table as it's not in the scraped static HTML code. It gets loaded in the browser only after we load the page, via javascript. Nonetheless, this is how we'd scrape the "a href" tag of class "indexList_link" if we had a static webpage with links:

In [7]:
def getAllLinks(link):
    soup = getSoup(link)

    a_elements = soup.findAll('a',{'class':'indexList_link'}) # class "indexList_link" contains the href link we desire
                
    return ['https://www.runczech.com/' + a.find('a')['href'] for a in a_elements] # list comprehension/function for links
    
marathons_links = getAllLinks('https://www.runczech.com/srv/www/qf/en/ramjet/results/list?&page=1&per_page=15')

In [8]:
marathons_links

[]

No such links can be found in output as it's loaded via javascript dynamically and we're scraping statically.

### Scrape a single table

Just an outline of how it could be coded in a static html page follows:

In [ ]:
def getNames(link):
    soup = getSoup(link)

    trs = soup.findAll('tr') # "tr" table-row element tag
    
    el = ['https://www.runczech.com/' + tr.find('td',{'class':'hidden980'}) for tr in trs] # hidden980 is class of first column
    sibling = el.next_sibling
    
#     return sibling.next_sibling # names are next to sibling of hidden980 class element/column
    return soup
    
names = getNames('https://www.runczech.com/srv/www/qf/en/ramjet/resultsEventDetail?eventId=21429&frm.subeventId=21430&page=1&per_page=15&sort=finishTime')

In [ ]:
names

In [ ]:
def getTimes(link):
    soup = getSoup(link)

    trs = soup.findAll('tr') # "tr" table-row element tag
    
    el = ['https://www.runczech.com/' + tr.find('td',{'class':'hidden767'}) for tr in trs] # hidden767 is class of "Chip time" column
    
#     return el
    return soup
    
times = getTimes('https://www.runczech.com/srv/www/qf/en/ramjet/resultsEventDetail?eventId=21429&frm.subeventId=21430&page=1&per_page=15&sort=finishTime')

In [ ]:
times

### HELP - scraping javascript dynamic website
 - https://www.google.com/search?q=python+scrape+website+that+has+script+inside+html&oq=python+scrape+website+that+has+script+inside+html&aqs=chrome..69i57.14882j0j7&sourceid=chrome&ie=UTF-8
     - https://stackoverflow.com/questions/26680590/how-to-scrape-imbeded-script-on-webpage-in-python
     - https://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Scraping_a_Webpage_Rendered_by_Javascript_Using_Python.php
     - https://www.youtube.com/watch?v=FSH77vnOGqU
     - https://www.youtube.com/watch?v=vsmxMLmroyQ

### Reworking the above static scraping into JS scraping: